<center><h1>     tiktok </h1></center>

<center><img src="https://yt3.ggpht.com/ytc/AKedOLRcwXL5heetbKNzpLCY3LOgml-72EcmpALhcvry5g=s900-c-k-c0x00ffffff-no-rj" alt="Logo" width="80" height="80"></center>

<center><h3> Part II: Data Cleaning/Preprocessing I </h3></center>

```diff
- TO DO:

- 1. steal preprocessing steps from modeling notebook

- 1. Additional info in markdown for each step.

- 2. Copy/paste background info from data collec file.


@ TO DO: EVENTUALLY

@ 1. Move all this to a script.

@ 2. Move second requests pull here and add to data collection process.

@ 3. Elim other columns earlier on if not needed.


+ QUESTIONS

+ think CV might be stripping emojis even though strip_accents = false

+ CV question = ?


```

#### Background

In [ ]:
# Copy/Paste some markdown from other file.

.

#### Imports

In [1]:
import numpy as np
import pandas as pd
import requests
import urllib
import os
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

.

### Data Cleaning - Single Country

Data Cleaning process for a single country first.

In [2]:
bz = pd.read_csv('./data/originals/tiktok_BZ.csv', index_col=[0])

In [3]:
bz.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 640 entries, 0 to 7
Data columns (total 37 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   region                                639 non-null    object 
 1   aweme_id                              640 non-null    int64  
 2   create_time                           640 non-null    int64  
 3   desc                                  562 non-null    object 
 4   author.uid                            640 non-null    int64  
 5   author.nickname                       640 non-null    object 
 6   author.signature                      599 non-null    object 
 7   author.custom_verify                  83 non-null     object 
 8   author.commerce_user_level            640 non-null    int64  
 9   is_ads                                640 non-null    bool   
 10  share_url                             640 non-null    object 
 11  statistics.play_count

.

In [4]:
bz.head(5)

,region,aweme_id,create_time,desc,author.uid,author.nickname,author.signature,author.custom_verify,author.commerce_user_level,is_ads,share_url,statistics.play_count,statistics.comment_count,statistics.share_count,statistics.download_count,statistics.digg_count,music.id,music.mid,music.source_platform,music.id_str,music.title,music.author,music.owner_id,music.owner_nickname,music.is_original,music.is_commerce_music,music.play_url.uri,music.strong_beat_url.uri,music.is_author_artist,music.is_pgc,music.matched_pgc_sound.author,music.matched_pgc_sound.title,music.matched_pgc_sound.mixed_title,music.matched_pgc_sound.mixed_author,music.external_song_info,music.binded_challenge_id,bodydance_score
0,BZ,7100638003651005701,1653246119,#fypシ @phoxxlee7 🥺,6773988713171354630,Jamie here,thank you all for 36k🇧🇿+501 feel free to ask a...,NaN,0,False,https://m.tiktok.com/v/7100638003651005701.htm...,50721,41,25,7,4554,7084947474011540229,7084947474011540229,72,7084947474011540229,original sound - d1vine.0,★,6.917312e+18,★,False,True,https://sf16-ies-music-va.tiktokcdn.com/obj/mu...,https://sf16-ies-music-sg.tiktokcdn.com/obj/ti...,False,False,NaN,NaN,NaN,NaN,[],0,0
1,BZ,7100977991836159237,1653325278,#fyp,7045711188025508869,shamarie lamb 🇧🇿,Starting Over,NaN,0,False,https://m.tiktok.com/v/7100977991836159237.htm...,66085,38,103,213,4421,7091124261821205254,7091124261821205254,72,7091124261821205254,BIG MOOD,💀,7.077681e+18,💀,False,True,https://sf16-ies-music-va.tiktokcdn.com/obj/mu...,https://sf16-ies-music-sg.tiktokcdn.com/obj/ti...,False,False,NaN,NaN,NaN,NaN,[],0,0
2,US,7086872101478583594,1650040994,#fyp,7010481384334656517,KAyyLee 🥰🥴,$kayylee66,NaN,0,False,https://m.tiktok.com/v/7086872101478583594.htm...,16671095,12374,51670,2299,748394,7086872064967232302,7086872064967232302,72,7086872064967232302,original sound - kayleehaynes970,KAyyLee 🥰🥴,7.010481e+18,KAyyLee 🥰🥴,False,True,https://sf16-ies-music-va.tiktokcdn.com/obj/ie...,NaN,False,False,NaN,NaN,NaN,NaN,[],0,0
3,FR,7081327470636453126,1648750034,"I tried, it worked 👟",6752451240984511494,Saafiir,Surprise on my Instagram \n🇩🇿🇪🇬\nSaafiirmanage...,NaN,0,False,https://m.tiktok.com/v/7081327470636453126.htm...,139811743,84590,112368,7252,12863104,6543230635689907200,6543230635689907200,72,6543230635689907200,original sound - taytay6737,Tay,6.542248e+18,Tay,False,True,https://sf16-ies-music-va.tiktokcdn.com/obj/mu...,https://sf16-ies-music-sg.tiktokcdn.com/obj/ti...,False,False,Justin Bieber,Baby,original sound - taytay6737 (Contains music fr...,"Contains music from: ""Baby""",[],0,0
4,CA,7091691923114642694,1651163197,“Peace. Too much fighting” — 🥺❤️ (Donation L1N...,6740960667346404357,Zachery Dereniowski,📧 zachery@onefluent.com\nEverything You Need ⬇️,Verified account,0,False,https://m.tiktok.com/v/7091691923114642694.htm...,23651468,15809,13280,25560,2353768,7091691909663542022,7091691909663542022,72,7091691909663542022,original sound - mdmotivator,Zachery Dereniowski,6.740961e+18,Zachery Dereniowski,False,True,https://sf16-ies-music-va.tiktokcdn.com/obj/mu...,https://sf16-ies-music-sg.tiktokcdn.com/obj/ti...,False,False,NaN,NaN,NaN,NaN,[],0,0


.

Filling NaNs in description field with a space so the hashtags can be countvectorized correctly later.

In [244]:
bz['desc'] = np.where(bz['desc'].isnull(), " ", bz['desc'])

.

Importing country dictionary for names/country codes.

In [9]:
import csv

codes = {}
with open("./data/codes.csv", mode='r') as inp:
    reader = csv.reader(inp)
    codes = {rows[1]:rows[0] for rows in reader}
    
codes.pop('Code')

'Name'

In [10]:
codes['BZ']

'Belize'

.

Adding a location column, since the "Country" column is country of tiktok origin rather than FYP country.

In [247]:
bz['fyp_location']=codes['BZ']

.

Fill nulls in description with empty string so it can be countvectorized.

In [248]:
bz['desc'] = np.where(bz['desc'].isnull(), " ", bz['desc'])

.

Drop nulls for origin region. I don't want videos if I don't know where they're from

In [249]:
bz = bz.dropna(subset=['region'], axis=0)

.

Drop a column that isn't as helpful as I expected.

In [250]:
bz = bz.drop(columns='author.custom_verify')

.

Checking date range for FYP videos 

In [251]:
import time
onemonth = 2592000 #approx; close enough

noww = time.time()
print(f"Oldest video: {np.round((noww - bz['create_time'].min()) /onemonth, 1)} months old")
print(f"Newest video: {np.round((noww - bz['create_time'].max())/onemonth, 1)} months old")

Oldest video: 3.3 months old
Newest video: 0.4 months old


1 march 2022, 30 may 2022.

* Reasonable range for amount of data pulled, but only have trends over 3 month span.
* Pulling trends for longer will take a LOT more data.

.

For music where users have used a popular singer's song and reuploaded under their own name, I am changing the "title" and "Artist" to "mixed title" and "mixed artist".

In [252]:
bz['music.title'] = np.where(bz['music.matched_pgc_sound.mixed_title'].notnull(
), bz['music.matched_pgc_sound.mixed_title'], bz['music.title'])

In [253]:
# checking work
len(bz[bz['music.matched_pgc_sound.mixed_title'] == bz['music.title']]), len(
    bz[bz['music.matched_pgc_sound.mixed_title'].notnull()])

(125, 125)

.

In [254]:
bz['music.author'] = np.where(bz['music.matched_pgc_sound.mixed_author'].notnull(
), bz['music.matched_pgc_sound.mixed_author'], bz['music.author'])

In [255]:
# checking work
len(bz[bz['music.matched_pgc_sound.mixed_author'] == bz['music.author']]), len(
    bz[bz['music.matched_pgc_sound.mixed_author'].notnull()])

(125, 125)

.

Extracting hashtags and putting them in their own column.

In [256]:
# Extract hashtags

bz['hashtags'] = bz.desc.str.findall(r'#.*?(?=\s|$)').str.join(" ")

.

Only keeping the top 10 hashtags for each country.

In [257]:
cv = CountVectorizer(max_features=10, strip_accents=False)

.

Countvectorizing hashtags.

In [258]:
X = bz['hashtags']
X_cv = pd.DataFrame(cv.fit_transform(X).A,  columns=['hash_'+x for x in cv.get_feature_names_out()], index=bz.index)
    
for column in X_cv.columns:
    bz[column]=X_cv[column]

.

Checking work. Scroll all the way to the right to see the hashtags columns.

In [259]:
bz.head(4)

,region,aweme_id,create_time,desc,author.uid,author.nickname,author.signature,author.commerce_user_level,is_ads,share_url,statistics.play_count,statistics.comment_count,statistics.share_count,statistics.download_count,statistics.digg_count,music.id,music.mid,music.source_platform,music.id_str,music.title,music.author,music.owner_id,music.owner_nickname,music.is_original,music.is_commerce_music,music.play_url.uri,music.strong_beat_url.uri,music.is_author_artist,music.is_pgc,music.matched_pgc_sound.author,music.matched_pgc_sound.title,music.matched_pgc_sound.mixed_title,music.matched_pgc_sound.mixed_author,music.external_song_info,music.binded_challenge_id,bodydance_score,fyp_location,hashtags,hash_belize,hash_belizean,hash_belizeantiktokers,hash_foryou,hash_foryoupage,hash_funny,hash_fyp,hash_fypシ,hash_trending,hash_viral
0,BZ,7100638003651005701,1653246119,#fypシ @phoxxlee7 🥺,6773988713171354630,Jamie here,thank you all for 36k🇧🇿+501 feel free to ask a...,0,False,https://m.tiktok.com/v/7100638003651005701.htm...,50721,41,25,7,4554,7084947474011540229,7084947474011540229,72,7084947474011540229,original sound - d1vine.0,★,6.917312e+18,★,False,True,https://sf16-ies-music-va.tiktokcdn.com/obj/mu...,https://sf16-ies-music-sg.tiktokcdn.com/obj/ti...,False,False,NaN,NaN,NaN,NaN,[],0,0,Belize,#fypシ,0,0,0,0,0,0,0,1,0,0
1,BZ,7100977991836159237,1653325278,#fyp,7045711188025508869,shamarie lamb 🇧🇿,Starting Over,0,False,https://m.tiktok.com/v/7100977991836159237.htm...,66085,38,103,213,4421,7091124261821205254,7091124261821205254,72,7091124261821205254,BIG MOOD,💀,7.077681e+18,💀,False,True,https://sf16-ies-music-va.tiktokcdn.com/obj/mu...,https://sf16-ies-music-sg.tiktokcdn.com/obj/ti...,False,False,NaN,NaN,NaN,NaN,[],0,0,Belize,#fyp,0,0,0,0,0,0,1,0,0,0
2,US,7086872101478583594,1650040994,#fyp,7010481384334656517,KAyyLee 🥰🥴,$kayylee66,0,False,https://m.tiktok.com/v/7086872101478583594.htm...,16671095,12374,51670,2299,748394,7086872064967232302,7086872064967232302,72,7086872064967232302,original sound - kayleehaynes970,KAyyLee 🥰🥴,7.010481e+18,KAyyLee 🥰🥴,False,True,https://sf16-ies-music-va.tiktokcdn.com/obj/ie...,NaN,False,False,NaN,NaN,NaN,NaN,[],0,0,Belize,#fyp,0,0,0,0,0,0,1,0,0,0
3,FR,7081327470636453126,1648750034,"I tried, it worked 👟",6752451240984511494,Saafiir,Surprise on my Instagram \n🇩🇿🇪🇬\nSaafiirmanage...,0,False,https://m.tiktok.com/v/7081327470636453126.htm...,139811743,84590,112368,7252,12863104,6543230635689907200,6543230635689907200,72,6543230635689907200,original sound - taytay6737 (Contains music fr...,"Contains music from: ""Baby""",6.542248e+18,Tay,False,True,https://sf16-ies-music-va.tiktokcdn.com/obj/mu...,https://sf16-ies-music-sg.tiktokcdn.com/obj/ti...,False,False,Justin Bieber,Baby,original sound - taytay6737 (Contains music fr...,"Contains music from: ""Baby""",[],0,0,Belize,,0,0,0,0,0,0,0,0,0,0


.

In [260]:
bz.isnull().sum()

region                                    0
aweme_id                                  0
create_time                               0
desc                                      0
author.uid                                0
author.nickname                           0
author.signature                         41
author.commerce_user_level                0
is_ads                                    0
share_url                                 0
statistics.play_count                     0
statistics.comment_count                  0
statistics.share_count                    0
statistics.download_count                 0
statistics.digg_count                     0
music.id                                  0
music.mid                                 0
music.source_platform                     0
music.id_str                              0
music.title                               0
music.author                              0
music.owner_id                          202
music.owner_nickname            

The rest of the nulls are not particularly worrying. I am keeping most of these columns for their value in double checking analysis.

It wasn't possible to find clear cut descriptions of all these fields, so some inference was made based on json structure and surrounding fields.

These columns will be dropped before modeling.

.

In [5]:
import spacy
en = spacy.load('en_core_web_sm')
stopwords = en.Defaults.stop_words

extra_words = ['fy', 'fyp', 'foryoupage', 'fypage', 'foryou', 'fypシ', 'viral', 'trending',
                                                              'trendingvideo', 'viralvideo', 'tiktok', 'like', 'don',
                            'lol', 'follow', 'time', 'day', 'replay', 'video', 'stop', 'want', 'got', 'best', 'love',
                            'stitch', 'reply', 'wait', 'come', 'bio', 'let', 'right', 'left', 'guy', 'guys', 'fun', 'things', 'end',
                            'going', 'year', 'know', 'good', 'new', 'll', 'need', 'happy', 'way', 'tho', 'help', 'today', 'days',
                            'live', 'feel', 'me', 'link', 'tho', 'nah', 'fyppppppppppppppppppppppp', 'fypp', 'find',  'gone',
                            'gotta', 'gonna', 'goviral', 'aint', 'yall', 'hi', 'heart', 'hear', 'hours', 'join', 'late', 'post', 'pls',
                            'st', 'tok', 'oh', 'ok', 'okay', 'said', 'sad', 'rip', 'yes', 'no', 'ya', 'yeah', 'yea', 'weeks', 'viraltiktok',
                            'tell', 'thank', 'ur', 'your', 'im', 'dis', 'idk', 'og', 'ib', 'ig', 'ob', 'iykyk', 'lmao', 'lmfao',
                            'link', 'likes', 'quick', 'seen', 'sure', 'tag', 've', 'm', 'plz', 'soon', 'isn', 'went', 'go', 'vs',
                            'have', 'having', 'happen', 'happened', 'go', 'goes', 'com', 'bit', 'bc', 'ad', 'ai', 'far', 'fave',
                             'fav', 'friend', 'great', 'guess', 'hard', 'held', 'hit', 'hold', 'leg', 'later', 'leave', 'left', 'life', 
                             'lot', 'little', 'look', 'long', 'lil', 'meet', 'miss', 'nice', 'nd', 'old', 'page', 'point', 'possible', 'pt',
                             'ready', 'real', 'set', 'st', 'start', 'stop', 'small', 'big', 'thing', 'think', 'tik', 'till', 'tip', 'talk',
                             'true', 'tt', 'try', 'turn', 'right', 'wrong', 'ago', 'bestie', 'bro', 'damn', 'vid', 'man', 'foryourpage',
                               'edit', 'xyzbca', 'zyxcba', 'zyzz', 'a', 'ab', 'abc', 'check', 'podcast', 'podcasts', 'share', 'walk', 'walks',
                               'walked'
                            ]

extra_hash = ['#'+i for i in extra_words]


print(len(stopwords))
stopwords = stopwords.union(extra_words).union(extra_hash)

326


#### Data Cleaning - All Countries

Since I have a lot of csvs to work with, I wrote a function to repeat the above data cleaning and preprocessing steps for all countries.

In [6]:
def dataclean(csv):
    """ change this to a docstring instead of comments every line """
    
    # Get country codes
    countrycode = csv[7:9]
    
    filename_to_read = (f'./data/originals/{csv}')
    
    filename_to_write = (f'./data/clean/c_{csv}')
    
    df = pd.read_csv(filename_to_read, index_col=[0])
    
    #add location
    df['fyp_location']=codes[countrycode]
    
    # fill nulls in description with empty string
    df['desc'] = np.where(df['desc'].isnull(), " ", df['desc'])
    
    # drop nulls for origin region
    df = df.dropna(subset=['region'], axis=0)
    
    # drop cols 
    df = df.drop(columns='author.custom_verify')

    # check dates
    # import time
    # onemonth = 2592000

    #noww = time.time()
    #print(f"Oldest video: {np.round( (noww - bz['create_time'].min()) /onemonth, 1)} months old")
    #print(f"Newest video: {np.round( (noww - bz['create_time'].max())/onemonth, 1)} months old")
    
    # Fix mix artists
    df['music.title'] = np.where(df['music.matched_pgc_sound.mixed_title'].notnull(
    ), df['music.matched_pgc_sound.mixed_title'], df['music.title'])
    
    df['music.author'] = np.where(df['music.matched_pgc_sound.mixed_author'].notnull(
    ), df['music.matched_pgc_sound.mixed_author'], df['music.author'])
    
    # get hastags
    df['hashtags'] = df.desc.str.findall(r'#.*?(?=\s|$)').str.join(" ")

    cv = CountVectorizer(max_features=10, strip_accents=False)
    X = df['hashtags']
    X_cv = pd.DataFrame(cv.fit_transform(X).A,  columns=['hash_'+x for x in cv.get_feature_names_out()], index=df.index)
    #scikit learn transformer returns numpy array, loses index. Reassign
    
    for column in X_cv.columns:
        df[column]=X_cv[column]
    
    df.to_csv(filename_to_write)
    print(f'exported{filename_to_write}')

In [7]:
files = [i for i in os.listdir('./data/originals') if not i.startswith('.')]

In [11]:
for file in files:
     dataclean(file)

exported./data/clean/c_tiktok_KI.csv
exported./data/clean/c_tiktok_HR.csv
exported./data/clean/c_tiktok_JM.csv
exported./data/clean/c_tiktok_FO.csv
exported./data/clean/c_tiktok_ET.csv
exported./data/clean/c_tiktok_EC.csv
exported./data/clean/c_tiktok_KH.csv
exported./data/clean/c_tiktok_DE.csv
exported./data/clean/c_tiktok_GI.csv
exported./data/clean/c_tiktok_GH.csv
exported./data/clean/c_tiktok_IT.csv
exported./data/clean/c_tiktok_JO.csv
exported./data/clean/c_tiktok_HT.csv
exported./data/clean/c_tiktok_GL.csv
exported./data/clean/c_tiktok_EE.csv
exported./data/clean/c_tiktok_GM.csv
exported./data/clean/c_tiktok_FI.csv
exported./data/clean/c_tiktok_ER.csv
exported./data/clean/c_tiktok_KY.csv
exported./data/clean/c_tiktok_HU.csv
exported./data/clean/c_tiktok_IQ.csv
exported./data/clean/c_tiktok_PF.csv
exported./data/clean/c_tiktok_IS.csv
exported./data/clean/c_tiktok_ID.csv
exported./data/clean/c_tiktok_FK.csv
exported./data/clean/c_tiktok_EG.csv
exported./data/clean/c_tiktok_GY.csv
e

.

.

.

.

.

.

.

.

In [268]:
# pattern matching for pgc_sounds could be done with timeseries function - forget what it's called. diff or lapse ?


# one lapse col for all pitches, concat all prev pitches

# one lapse col for all beats, concat all prev beats

# or just put in neural net probably makes more sense. predict if pgc match is found for sound 

Additional data collection 

USA only?

In [ ]:
pitch_li = []
for x in bz['music.strong_beat_url.uri']:
    try:
        df = pd.json_normalize(json.load(urllib.request.urlopen(x)),
                               record_path='pitchs').T

        df['url'] = x

        df.columns = [str(i) + '_pitch' for i in df.columns]

        pitch_li.append(df)
    except:
        pass

pitches = pd.concat(pitch_li)
pitches.head(4)

#

bz = pd.merge(left=bz, right=pitches, how='left',
              left_on='music.strong_beat_url.uri', right_on='url_pitch')

# Due to insane number of beats collected for some songs I will do this analysis for US only.



onset_li = []
for x in bz['music.strong_beat_url.uri']:
    try:
        df = pd.json_normalize(json.load(urllib.request.urlopen(x)),
                               record_path='onsets').T

        df['url'] = x

        df.columns = [str(i) + '_onset' for i in df.columns]

        onset_li.append(df)
    except:
        pass

onsets = pd.concat(onset_li)
bz = pd.merge(left=bz, right=onsets, how='left',
              left_on='music.strong_beat_url.uri', right_on='url_onset')



duration_li = []
for x in bz['music.strong_beat_url.uri']:
    try:
        df = pd.json_normalize(json.load(urllib.request.urlopen(x)),
                               record_path='durations').T

        df['url'] = x

        df.columns = [str(i) + '_duration' for i in df.columns]

        duration_li.append(df)
    except:
        pass

durations = pd.concat(duration_li)
bz = pd.merge(left=bz, right=durations, how='left',
              left_on='music.strong_beat_url.uri', right_on='url_duration')

# Due to insane number of beats collected for some songs I will do this analysis for US only.

# Due to insane number of beats collected for some songs I will do this analysis for US only.



In [ ]:
bz = pd.merge(left=bz, right=pitches, how='left',
              left_on='music.strong_beat_url.uri', right_on='url_pitch')

In [ ]:
onset_li = []
for x in bz['music.strong_beat_url.uri']:
    try:
        df = pd.json_normalize(json.load(urllib.request.urlopen(x)),
                               record_path='onsets').T

        df['url'] = x

        df.columns = [str(i) + '_onset' for i in df.columns]

        onset_li.append(df)
    except:
        pass

onsets = pd.concat(onset_li)
bz = pd.merge(left=bz, right=onsets, how='left',
              left_on='music.strong_beat_url.uri', right_on='url_onset')

# Due to insane number of beats collected for some songs I will do this analysis for US only.

In [ ]:
duration_li = []
for x in bz['music.strong_beat_url.uri']:
    try:
        df = pd.json_normalize(json.load(urllib.request.urlopen(x)),
                               record_path='durations').T

        df['url'] = x

        df.columns = [str(i) + '_duration' for i in df.columns]

        duration_li.append(df)
    except:
        pass

durations = pd.concat(duration_li)
bz = pd.merge(left=bz, right=durations, how='left',
              left_on='music.strong_beat_url.uri', right_on='url_duration')

# Due to insane number of beats collected for some songs I will do this analysis for US only.

In [ ]:
# was trying to pull out strong beat info, may be too hard.

In [ ]:

# pd.json_normalize(data, record_path='durations').T
# pd.json_normalize(data, record_path='onsets').T
# pd.json_normalize(data, record_path='pitchs').T

In [ ]:
bz['music.strong_beat_url.uri'].isna().sum()

In [ ]:
# change json url to actual json. should have done in data acq step but i am not going back.

In [ ]:
for x in bz['music.strong_beat_url.uri']:
    try:
        bz['music.strong_beat_analysis'] = json.load(urllib.request.urlopen(x))
    except:
        pass

In [ ]:
bz.head(5)

In [ ]:
request = json.load(urllib.request.urlopen(bz['music.strong_beat_url.uri']))
data = json.load(request)
pd.json_normalize(data, record_path='durations').T


# new_cols = ['column_new_1', 'column_new_2', 'column_new_3']
# new_vals = [np.nan, 'dogs', 3]
# df = df.reindex(columns=df.columns.tolist() + new_cols)   # add empty cols
# df[new_cols] = new_vals  # multi-column assignment works for existing cols

In [106]:
import csv

codes = {}
with open("./data/codes.csv", mode='r') as inp:
    reader = csv.reader(inp)
    codes = {rows[1]:rows[0] for rows in reader}
    
codes.pop('Code')

'Name'